# Overview
This Jupyter notebook helps you create a 3D model using the Photoscan photogrammetry software, leveraging the computing power of a high-performance compute cluster. It assumes that your photographs are stored in a folder on [Box](http://berkeley.box.com), and the results of each step are saved back to Box.

If you want to manually edit the results of any step in the process, you can sync or download the project file and photos from Box to any computer with Photoscan installed (version 1.5 or higher), make your edits, and move the changes back to Box before proceeding with the next computationally-intensive processing step.

### Software versions
This notebook has been tested using:
* python 3.5 kernel
* boxsdk (2.0.0a2)
* Photoscan 1.3 or later

## Workflow at a glance
The photogrammetry workflow laid out here involves the following steps. Each listed step has its own section in the notebook. 
1. **Overview**: the overview that you're currently reading
2. **Source Files**: defines where various source files are located.
3. **Box Setup**: connects this notebook to Box so it can access the photos you've stored there. 
4. **Project Setup**: pulls your images into the computing cluster, and creates a new Photoscan (psx) file if needed.
5. **Align Photos**: arranges the photos you've taken, relative to one another. 1% to 8% of the expected processing time.
6. **Build Dense Cloud**: creates the a point cloud from the photographs. The most computationally-intensive step, which can take 50%-96% of the total model generation time, depending on your settings. This step is run using GPU nodes on the cluster, which improve performance.
7. **Build Mesh**: connecting the point cloud together to build a mesh takes around 30% of the processing time on lower quality settings, or as little as 2% on higher quality settings.
8. **Build Texture**: building the texture takes between 8% (medium quality) and 1% (ultra high quality) of the processing time.
9. **Export Results**: exporting the photomosaic and OBJ files, and moving them to Box.

# Project Setup
This step connects the notebook to Box. Before you get started, make sure you have a folder in Box for your project. That folder should have a folder inside it called "images" that has all the photos you want to use for your 3D model. This workflow will also create a new folder, "files", that contains the project files-- you don't have to set that up in advance.

Click the code box below, and then click the "run cell" button in the toolbar. This will run the code and move to the next box.

After you run the next code box, your user name on the cluster should appear at the bottom.

In [78]:
# widget imports
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, VBox
import ipywidgets as widgets
from IPython.display import display

usernamearray = %sx whoami
username = usernamearray[0]
print('user name: ', username)

def f(x):
    return x

def pn(projectName):
    return projectName

def smf(mountFolder):
    return mountFolder

def bpf(boxFolder):
    return boxFolder

def scp(containerPath):
    return containerPath

def idd(imageDataPath):
    return imageDataPath

def hfp(homePath):
    return homePath

def rf(runPath):
    return runPath


#Define widgets and defaults
pntextbox=widgets.Text(value='projectname', description='Project Name')
#singularitymountfoldertextbox =widgets.Text(value='/scratch/', description='Singularity Mount Folder') 
singularitymountfoldertextbox =widgets.Text(value='/scratch/') 
boxfoldertextbox =widgets.Text(value='myBoxFolderName', description='Box Project Folder') 
scptextbox=widgets.Text(value='/global/scratch/groups/dh/photoscan13.img', description='Path to Container')
#iddtextbox=widgets.Text(value='/global/scratch/' + username + '/imageData', description='Image Data')
hometextbox=widgets.Text(value='/global/home/users/' + username + '/', description='Home Folder')
runtextbox=widgets.Text(value='/global/scratch/' + username + '/'  + pntextbox.value + '/', description='Run Folder')

user name:  quinnd


## Project set up
After you run the code box below, a set of text boxes will appear. Click in the text boxes and enter the following values:
* **Project name**: The name of your project. Will be used for some of the folders on the cluster that will be created to process the project.
* **Mount folder**: Where your photos will be uploaded for processing. Unless you know specifically this should be somewhere else, leave this as /scratch/
* **Box project folder**: The name of the folder on Box that has your photos in it.
* **Path to container**: The location on the cluster where the container with the Photoscan software is installed. At Berkeley, this container is at /global/scratch/groups/dh/photoscan13.img.
* **Home folder**: Your home folder on the cluster. 
* **Run folder**: The folder where the various job scripts to trigger the processing steps will be stored. Default: /global/scratch/your-user-name/your-project-name/

When you're done filling in the text boxes, hit the "run cell" button again.


In [116]:
# Enter the projectname
interact(pn, projectName=pntextbox)

# Enter the Singularity Mount Folder here, this is defined inside the Singularity container:
interact(smf, mountFolder=singularitymountfoldertextbox)

# Enter the Box Project folder
interact(bpf, boxFolder=boxfoldertextbox)

# Enter the path to the Singularity container hosting Photoscan
interact(scp, containerPath=scptextbox)

# Enter the path to where the iamge files from Box should be place on the scrath drive
#interact(idd, imageDataPath=iddtextbox)

# Enter the path to the user home folder where Box oauth information is stored in files.
interact(hfp, homePath=hometextbox)

# Enter the path to the run home folder where processing will be conducted and intermediate files created.
interact(rf, runPath=runtextbox)

'/global/scratch/quinnd/monument/'

<function __main__.rf>

## Creating project folders
The code box below creates the project folders (e.g. run folder, scratch images directory) on the cluster if they don't already exist.

In [105]:
projectname =  pntextbox.value

# Defines all paths and filenames
import os

usernamearray = %sx whoami
username = usernamearray[0]
print('user name: ', username)
print('project name: ', projectname)

runFolder = os.path.join(runtextbox.value, '')
homeFolder =  os.path.join(hometextbox.value, '')
print('run folder: ', runFolder)

# Creates the run folder if it does not exist
if ( not os.path.isdir(runFolder)):
    print('run folder does not exist, creating.')
    os.mkdir(runFolder)

singularitymountfolder = os.path.join(singularitymountfoldertextbox.value, '')

boxProjectFolder = boxfoldertextbox.value
projectFile = '/' + projectname + '.psx'
datazipFile = '/' + projectname + '.files.zip'
dataFolder = '/' + projectname + '.files'

projectfileid = ''
jpgFile = projectname + '.jpg'

scratchImageDataDirectory = runFolder + 'images/'
print('scratchImageDataDirectory: ', scratchImageDataDirectory)
# Creates the image folder if it does not exist
if ( not os.path.isdir(scratchImageDataDirectory)):
    print('images folder does not exist, creating.')
    os.mkdir(scratchImageDataDirectory)

slurmScript = runFolder + 'slurmscript.sh'
execScript = runFolder + 'execscript.sh'
scratchExecScript = singularitymountfolder + 'execscript.sh'
commandScript = runFolder + 'commandscript.sh'

singularityContainerPath = scptextbox.value
#/global/scratch/groups/dh/photoscan13.img

user name:  quinnd
project name:  monument
run folder:  /global/scratch/quinnd/monument/
scratchImageDataDirectory:  /global/scratch/quinnd/monument/images/


## Box setup

To connect Box with this notebook, you need to go through some one-time configuration steps the first time you use this notebook for any project. Please run the *BoxAuthenticationBootstrap.ipynb* notebook, then return to the code cells below to finish setup. After you finish the last cell, you should see *Folder name:  All Files*.

If you run this notebook again for another project, you won't have to run *BoxAuthenticationBootstrap.ipynb* again, and can just continue to the code below.

In [106]:
def store_tokens(access_token, refresh_token):
    
    """Callback for storing refresh tokens. (For now we ignore access tokens)."""
    with open(homeFolder + 'apptoken.cfg', 'w') as f:
     f.write(refresh_token.strip())

In [107]:
import os

CLIENT_ID = None
CLIENT_SECRET = None
REDIRECT_URI = None
# Read app info from text file
with open(homeFolder + 'app.cfg', 'r') as app_cfg:
    CLIENT_ID = app_cfg.readline()
    CLIENT_SECRET = app_cfg.readline()
    REDIRECT_URI = app_cfg.readline()


In [108]:
REFRESH_TOKEN = None

# Read app info from text file
with open(homeFolder + 'apptoken.cfg', 'r') as apptoken_cfg:
    REFRESH_TOKEN = apptoken_cfg.readline()

In [109]:
from boxsdk import OAuth2
from boxsdk import Client

# Do OAuth2 authorization.
oauth = OAuth2(
    client_id=CLIENT_ID.strip(),
    client_secret=CLIENT_SECRET.strip(),
    refresh_token=REFRESH_TOKEN.strip(),
    store_tokens=store_tokens
)

client = Client(oauth)

root_folder = client.folder(folder_id='0').get()
print ("Folder name: ", root_folder['name'] )

items = client.folder(folder_id='0').get_items(limit=100, offset=0)
#print ("items: ", items )



Folder name:  All Files


## Finding folder on box
The function below looks for your project folder on Box.

In [110]:
def find_folder_id(folder_name):
    print ('find_folder_id folder_name: ', folder_name)
    folderlist = client.search(query=folder_name, result_type='folder', limit=100, offset=0)
    
    print ('find_folder_id folderlist: ', folderlist)
    
    if len(folderlist) == 0:
        print('folder not found: ', folder_name)
        return 0
    else:
        for fldr in folderlist:
            if fldr['name'] == folder_name :
                return fldr['id']
            
        return 0

## Create command scripts template
This code writes out a execute file that includes the license install as the first step each time to enable any HPC node. This is a temporary workaround until release 1.3 is installed on Savio and a license server is configured.

In [111]:
# script for singularity to run
import os, stat

execScriptTemplate = '/opt/photoscan-pro/photoscan.sh --activate TBPD4-ACEN7-ZOM2L-TY3L8-PEP3G\n\
/opt/photoscan-pro/photoscan.sh -r  /scratch/commandscript.sh \n'


with open(execScript, 'w') as f:
    f.write(execScriptTemplate)
    
os.chmod(execScript,  0o755)    


In [112]:
# batch script
batchtemplate = '#!/bin/bash  \n\
# Job name: \n\
#SBATCH --job-name=' + projectname + '\n\
# \n\
# Account: \n\
#SBATCH --account=ac_scsguest \n\
# \n\
# Partition: \n\
#SBATCH --partition=savio \n\
# \n\
# Wall clock limit: \n\
#SBATCH --time={} \n\
# \n\
## Command(s) to run: \n\
singularity exec --writable -B ' + runFolder + ':' + singularitymountfolder + '  ' + singularityContainerPath + '  ' + scratchExecScript + '\n' 
  
    

## Function for updating project file
Multiple times in the workflow, the project folder needs to be updated on Box in order to enable making edits through the desktop version of Photoscan. This sets up a utility funcation that pushes the project 
file into the working directory on Savio. It assumes that the project folder will only contain one project file and one zipped archive.

In [113]:
import os
import shutil

def update_project_file_in_box():

    newFolderId = find_folder_id(boxProjectFolder)
    print ("Box folder id: ", newFolderId  )
    
    # create a zipped archive of the data folder
    shutil.make_archive(runFolder + dataFolder, 'zip', runFolder + dataFolder)
    
    projectfilelist = client.search(query=projectFile, result_type='file', limit=10, 
                                offset=0, ancestor_folders=[client.folder(folder_id=newFolderId)],
                                file_extensions=['psx'] )  
    
    print ("project file list: ", projectfilelist  )
    
    # if project file is not yet in folder, upload both the project file and the data zip file
    if len(projectfilelist) == 0 or len(projectfilelist) > 1: 
        print ("project file not found. " )
        upload_folder = client.folder(folder_id=newFolderId).get()
        projectpsx = upload_folder.upload(runFolder + projectFile)  
        print ("project file id: ", projectpsx['id'] )
        projectzip = upload_folder.upload(runFolder + datazipFile)  
        print ("data zip file id: ", projectzip['id'] )
        return
    else:
        datazipfilelist = client.search(query=datazipFile, result_type='file', limit=10, 
                                offset=0, ancestor_folders=[client.folder(folder_id=newFolderId)])
                                #file_extensions=['zip'] )  
        print ("datazip file list: ", datazipfilelist  )
        
        projectfileid = projectfilelist[0]['id']
        print ("project file id: ", projectfileid )
        datazipfileid = datazipfilelist[0]['id']
        print ("data zip file list: ", datazipfileid  )
    
        update_file = client.file(file_id=projectfileid).get()
        update_zip_file = client.file(file_id=datazipfileid).get()

        # upload a new version of the project file
        print ('begin project file update.' )
        psxfile = update_file.update_contents(runFolder + projectFile)  
        zipfile = update_zip_file.update_contents(runFolder + datazipFile)
        print ('update psx result: ', psxfile ,'   update zip result: ', zipfile )

## Fetching images from Box
The code below retrieves the images from your project directory on Box, and puts them in the project folder on the cluster that you specified above.

When you run it, it will check for the number of files in the folder on Box, and list each of them as it downloads. Please wait until you see *Download complete* before moving to the next step -- this can take some time if you have a lot of images, or large images.

In [114]:
import os
import shutil 


os.chdir(scratchImageDataDirectory)
print ('retrieving images from folder: ', boxProjectFolder)
print ('downloading images to directory: ', scratchImageDataDirectory)

# test folder contents
items = client.folder(folder_id='0').get_items(limit=20, offset=0)
if type(items) is list:
    #print ('number of files in top folder: ', len(items) )
    
    targetfolderId = ''
    for item in items:
        if item['type'] == 'folder':
            #print('folder name: ', item['name'])
            if item['name'] == boxProjectFolder:
                targetfolderId = item['id']
                print('targetfolderId: ', targetfolderId)
        
    if targetfolderId is not None:
        tgtitems = client.folder(folder_id=targetfolderId).get_items(limit=200, offset=0)
        if type(tgtitems) is list:
            print ('number of files in target folder: ', len(tgtitems) ) 
        
        # download all image files
        for tgtitem in tgtitems:
            if  not tgtitem['type'] == 'folder' and tgtitem['name'].endswith('.JPG'):
                print('downloading: ', tgtitem['name'])
                imagecontent = client.file(file_id=tgtitem['id']).content()
                newfile = open(scratchImageDataDirectory + tgtitem['name'], 'wb')
                newfile.write(imagecontent)
                newfile.close()
        print ('Download complete')

retrieving images from folder:  monument
downloading images to directory:  /global/scratch/quinnd/monument/images/
targetfolderId:  27697792189
number of files in target folder:  38
downloading:  IMG_3023.JPG
downloading:  IMG_3024.JPG
downloading:  IMG_3025.JPG
downloading:  IMG_3026.JPG
downloading:  IMG_3027.JPG
downloading:  IMG_3028.JPG
downloading:  IMG_3029.JPG
downloading:  IMG_3030.JPG
downloading:  IMG_3031.JPG
downloading:  IMG_3032.JPG
downloading:  IMG_3033.JPG
downloading:  IMG_3034.JPG
downloading:  IMG_3035.JPG
downloading:  IMG_3036.JPG
downloading:  IMG_3037.JPG
downloading:  IMG_3038.JPG
downloading:  IMG_3039.JPG
downloading:  IMG_3040.JPG
downloading:  IMG_3041.JPG
downloading:  IMG_3042.JPG
downloading:  IMG_3043.JPG
downloading:  IMG_3044.JPG
downloading:  IMG_3045.JPG
downloading:  IMG_3046.JPG
downloading:  IMG_3047.JPG
downloading:  IMG_3048.JPG
downloading:  IMG_3049.JPG
downloading:  IMG_3050.JPG
downloading:  IMG_3051.JPG
downloading:  IMG_3052.JPG
download

## Function for retrieving project file
This code sets up a function to bring an updated project file back from Box, e.g. after you've made changes using the desktop version of Photoscan, and you want to run a subsequent processing step on a cluster.

In [115]:

import os
import shutil 
def retrieve_project_file():
    newFolderId = find_folder_id(boxProjectFolder)
    tgtitems = client.folder(folder_id=newFolderId).get_items(limit=1000, offset=0)

    # download the project file
    for tgtitem in tgtitems:
        if  not tgtitem['type'] == 'folder' and tgtitem['name'].endswith('.psx'):
            print('downloading: ', tgtitem['name'])
            imagecontent = client.file(file_id=tgtitem['id']).content()
            newfile = open(runFolder + tgtitem['name'], 'wb')
            newfile.write(imagecontent)
            newfile.close()
            print('project file download complete.')
            
        if  not tgtitem['type'] == 'folder' and tgtitem['name'].endswith('.zip'):
            
            # delete the old folder
            !rm -rf $runFolder$dataFolder
            
            print('downloading: ', tgtitem['name'])
            imagecontent = client.file(file_id=tgtitem['id']).content()
            newfile = open(runFolder + tgtitem['name'], 'wb')
            newfile.write(imagecontent)
            newfile.close()
            
            shutil.unpack_archive(runFolder + datazipFile, runFolder + dataFolder ,'zip')
            
            print('data zip file download complete.')

# Project setup

This section pulls your images onto the computing cluster and creates and uploads a Photoscan .psx file if it doesn't already exist.



## Set up job script

This code sets up the job script that will be run on the cluster, in order to pull in your images.

In [15]:
def hrs(xhrs):
    return xhrs
def mins(xmins):
    return xmins

#Define widgets and defaults
hrslider=widgets.IntSlider(description='Estimated Hours:',  value=1, min=0, max=23,step=1)
minslider=widgets.IntSlider(description='Estimated Minutes:',  value=0, min=0, max=59,step=1)


In [16]:
# Enter the estimated hours and minutes to create the project and add photos
interact(hrs, xhrs=hrslider)
interact(mins, xmins=minslider)

0

<function __main__.mins>

In [17]:
import os, stat

files = [ singularitymountfolder + 'images/' + f for f in os.listdir(scratchImageDataDirectory) ] 

template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
import time \n\
doc = PhotoScan.app.document \n\
chunk = PhotoScan.app.document.addChunk() \n\
chunk.addPhotos( {} ) \n\
doc.save(path=\"{}\", chunks = [doc.chunk])\n '

output = template.format(str(files), singularitymountfolder + projectFile)
print('output: ', output)

with open(commandScript, 'w') as f:  
    f.write(output)

os.chmod(commandScript,  0o755)

#set time limit for this batch run
minstring = str(minslider.value)
if minslider.value < 10:
    minstring = '0' + str(minslider.value)
estimateTime = str(hrslider.value) + ':' + minstring + ':00'
print('estimateTime: ', estimateTime)
outputbatchscript = batchtemplate.format(estimateTime)
with open(slurmScript, 'w') as f:  
    f.write(outputbatchscript)
    
os.chmod(slurmScript,  0o755)

output:  #!/usr/bin/env python3 
import PhotoScan 
import time 
doc = PhotoScan.app.document 
chunk = PhotoScan.app.document.addChunk() 
chunk.addPhotos( ['/scratch/images/IMG_3045.JPG', '/scratch/images/IMG_3051.JPG', '/scratch/images/IMG_3052.JPG', '/scratch/images/IMG_3048.JPG', '/scratch/images/IMG_3054.JPG', '/scratch/images/IMG_3034.JPG', '/scratch/images/IMG_3036.JPG', '/scratch/images/IMG_3049.JPG', '/scratch/images/IMG_3053.JPG', '/scratch/images/IMG_3040.JPG', '/scratch/images/IMG_3027.JPG', '/scratch/images/IMG_3031.JPG', '/scratch/images/IMG_3028.JPG', '/scratch/images/IMG_3029.JPG', '/scratch/images/IMG_3024.JPG', '/scratch/images/IMG_3025.JPG', '/scratch/images/IMG_3038.JPG', '/scratch/images/IMG_3033.JPG', '/scratch/images/IMG_3042.JPG', '/scratch/images/IMG_3026.JPG', '/scratch/images/IMG_3043.JPG', '/scratch/images/IMG_3046.JPG', '/scratch/images/IMG_3044.JPG', '/scratch/images/IMG_3041.JPG', '/scratch/images/IMG_3050.JPG', '/scratch/images/IMG_3023.JPG', '/scratch/ima

## Run job
The code below will run the job on the cluster, pulling in your images from Box, and will display the job ID.

In [18]:
os.chdir(runFolder)

out = !sbatch slurmscript.sh   
    
print ('Execute image loading output: ', out ) 
jobId =  out[0].split()[3]
print (jobId)

Execute image loading output:  ['Submitted batch job 1406445']
1406445


In [19]:
# print the users queue and the job status by id
!squeue -u $username
print('--------------------------------')
!scontrol show job $jobId

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1406445     savio monument   quinnd  R       0:03      1 n0021.savio1
--------------------------------
JobId=1406445 JobName=monument
   UserId=quinnd(42413) GroupId=ucb(501) MCS_label=N/A
   Priority=989225 Nice=0 Account=ac_scsguest QOS=savio_normal
   JobState=RUNNING Reason=None Dependency=(null)
   Requeue=0 Restarts=0 BatchFlag=1 Reboot=0 ExitCode=0:0
   RunTime=00:00:03 TimeLimit=01:00:00 TimeMin=N/A
   SubmitTime=2017-06-10T06:21:02 EligibleTime=2017-06-10T06:21:02
   StartTime=2017-06-10T06:21:02 EndTime=2017-06-10T07:21:02 Deadline=N/A
   PreemptTime=None SuspendTime=None SecsPreSuspend=0
   Partition=savio AllocNode:Sid=ln000:21556
   ReqNodeList=(null) ExcNodeList=(null)
   NodeList=n0021.savio1
   BatchHost=n0021.savio1
   NumNodes=1 NumCPUs=20 NumTasks=1 CPUs/Task=1 ReqB:S:C:T=0:0:*:*
   TRES=cpu=20,mem=62.50G,node=1
   Socks/Node=* NtasksPerN:B:S:C=0:0:*:* CoreSpec=*
   MinCPU

## Upload project file
If there isn't a "data" subfolder inside your project folder on Box already, the code below will first create it. The code will then upload a new Photoscan project file and associated data files to Box. 

If a Photoscan project file with the same name already exists in the folder on Box, this code will *not* overwrite it.

Please note: New folders in Box can take several minutes to register, so this step may take a few minutes. If you get a "file not found" error in the next step, wait a minute or two and retry.

In [20]:
from boxsdk.exception import BoxAPIException

newFolderId = find_folder_id(boxProjectFolder)
print('folder id: ', newFolderId)

if newFolderId == 0:
    newFolder = client.folder(folder_id='0').create_subfolder(boxProjectFolder)
    newFolderId = newFolder['id']
    print ("folder created" )
else: 
    print ("folder exists" )
    
print ("folder id: ", newFolderId )

update_project_file_in_box()

find_folder_id folder_name:  monument
find_folder_id folderlist:  [<Box Folder - 27697792189 (monument)>]
folder id:  27697792189
folder exists
folder id:  27697792189
find_folder_id folder_name:  monument
find_folder_id folderlist:  [<Box Folder - 27697792189 (monument)>]
Box folder id:  27697792189
project file list:  [<Box File - 183058668349 (monument.psx)>]
datazip file list:  [<Box File - 183058591009 (monument.files.zip)>]
project file id:  183058668349
data zip file list:  183058591009
begin project file update.
update psx result:  <Box File - 183058668349>    update zip result:  <Box File - 183058591009>


## OFFLINE: Image masking
If you need to mask any images before building the dense cloud, please launch your desktop version of Photoscan, make those changes, and save the modified .psx file back to your folder on Box before proceeding with the next step.

# Align photos and Build Dense Cloud

The code below will retrieve the project file from Box. This will ensure that you're working with the latest version, even if you've made changes offline (such as image masking).

In [21]:
retrieve_project_file()

find_folder_id folder_name:  monument
find_folder_id folderlist:  [<Box Folder - 27697792189 (monument)>]
downloading:  monument.files.zip
data zip file download complete.
downloading:  monument.psx
project file download complete.


## Set up job script

Run the code box below, then choose your estimated run time for the job. The default run time is 3 hours, which should cover most projects with under 75 photos. If you have more photos, please increase the run time.

**Maurice: make this a different widget with default run time of 3 hours**

In [118]:
# Enter the estimated hours and minutes to create the project and add photos
interact(hrs, xhrs=hrslider)
interact(mins, xmins=minslider)

0

<function __main__.mins>

## Set up parameters
Choose the quality settings for the dense cloud build after running the cells below.

In [121]:
def chc(txt):
    return txt

pselchoice=widgets.Dropdown(description='Preselection', value='GenericPreselection', options=['GenericPreselection', 'ReferencePreselection'])
qualchoice=widgets.Dropdown(description='Quality', value='MediumQuality', options=['UltraQuality', 'HighQuality', 'MediumQuality',  'LowQuality', 'LowestQuality'] )
fltrchoice=widgets.Dropdown(description='Filter', value='ModerateFiltering', options=['NoFiltering', 'MildFiltering', 'ModerateFiltering',  'AggressiveFiltering']  )
accchoice=widgets.Dropdown(description='Accuracy', value='MediumAccuracy', options= ['HighestAccuracy', 'HighAccuracy', 'MediumAccuracy',  'LowAccuracy', 'LowestAccuracy']  )
afchoice=widgets.Dropdown(description='Adaptive Fitting', options=['True', 'False'])

In [122]:
interact(chc, txt=pselchoice)
interact(chc, txt=qualchoice)
interact(chc, txt=fltrchoice)
interact(chc, txt=accchoice)
interact(chc, txt=afchoice)

'True'

<function __main__.chc>

## Create job script
The next cell will create and display the job script that will be run on the cluster.

In [124]:
# Creates job script for building dense cloud
import os, stat

template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
import time \n\
print( "start time: ", time.time()) \n\
doc = PhotoScan.app.document \n\
doc.open(\"{}\") \n\
chunk = doc.chunk \n\
chunk.matchPhotos(accuracy=PhotoScan.{}, preselection=PhotoScan.{}) \n\
chunk.alignCameras(adaptive_fitting={}) \n\
chunk.buildDenseCloud(quality=PhotoScan.{},  filter=PhotoScan.{} ) \n\
doc.save(\"{}\") \n\
print( "stop time: ", time.time()) \n'

output = template.format(singularitymountfolder + projectFile, accchoice.value, pselchoice.value, afchoice.value, qualchoice.value, fltrchoice.value, singularitymountfolder + projectFile)

print('output: ', output)

with open(commandScript, 'w') as f:  
    f.write(output)

    
#Set time limit for this batch run
minstring = str(minslider.value)
if minslider.value < 10:
    minstring = '0' + str(minslider.value)
estimateTime = str(hrslider.value) + ':' + minstring + ':00'
print('estimateTime: ', estimateTime)
outputbatchscript = batchtemplate.format(estimateTime)
#outputbatchscript = batchtemplate.format('01:30:00')
with open(slurmScript, 'w') as f:  
    f.write(outputbatchscript)
    

output:  #!/usr/bin/env python3 
import PhotoScan 
import time 
print( "start time: ", time.time()) 
doc = PhotoScan.app.document 
doc.open("/scratch//monument.psx") 
chunk = doc.chunk 
chunk.matchPhotos(accuracy=PhotoScan.MediumAccuracy, preselection=PhotoScan.GenericPreselection) 
chunk.alignCameras(adaptive_fitting=True) 
chunk.buildDenseCloud(quality=PhotoScan.MediumQuality,  filter=PhotoScan.ModerateFiltering ) 
doc.save("/scratch//monument.psx") 
print( "stop time: ", time.time()) 

estimateTime:  1:00:00


## Run dense cloud build
Running the following code box will submit your job to the cluster. Your job ID will be printed below.

In [31]:
out = !sbatch slurmscript.sh   
    
print ('Execute the dense cloud build output: ', out ) 
jobId =  out[0].split()[3]
print (jobId)

Execute the dense cloud build output:  ['Submitted batch job 1406448']
1406448


## Check on job status
Building the dense cloud is the longest-running phase of the photogrammetry process. Depending on how many photos you have, and the quality settings you've chosen, the job could run for anywhere from minutes to hours. Run the following code box to check on the status of the dense cloud build. You can run it multiple times for new updates.

In [34]:
# print the users queue and the job status by id
!squeue -u $username
print('--------------------------------')
!scontrol show job $jobId

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
--------------------------------
JobId=1406448 JobName=monument
   UserId=quinnd(42413) GroupId=ucb(501) MCS_label=N/A
   Priority=735030 Nice=0 Account=ac_scsguest QOS=savio_normal
   JobState=FAILED Reason=NonZeroExitCode Dependency=(null)
   Requeue=0 Restarts=0 BatchFlag=1 Reboot=0 ExitCode=1:0
   RunTime=00:06:49 TimeLimit=01:00:00 TimeMin=N/A
   SubmitTime=2017-06-10T07:02:36 EligibleTime=2017-06-10T07:02:36
   StartTime=2017-06-10T07:02:38 EndTime=2017-06-10T07:09:27 Deadline=N/A
   PreemptTime=None SuspendTime=None SecsPreSuspend=0
   Partition=savio AllocNode:Sid=ln000:21556
   ReqNodeList=(null) ExcNodeList=(null)
   NodeList=n0044.savio1
   BatchHost=n0044.savio1
   NumNodes=1 NumCPUs=20 NumTasks=0 CPUs/Task=1 ReqB:S:C:T=0:0:*:*
   TRES=cpu=20,mem=62.50G,node=1
   Socks/Node=* NtasksPerN:B:S:C=0:0:*:* CoreSpec=*
   MinCPUsNode=1 MinMemoryNode=62.50G MinTmpDiskNode=0
   Features=(null) Gres=(

## Display log message
As an optional step, you can run the code below to see the log file once the job is done. It includes technical information about the points that were found in each photo, rectifying disparities between photos, etc.

In [35]:
# print the log file if necessary
slurmlogfilename = 'slurm-' + jobId + '.out'
with open(slurmlogfilename, 'r') as fin:
    print(fin.read(), end="")

Activation successful
start time:  1497103364.3941915
LoadProject
loaded project in 0.122341 sec
MatchPhotos: accuracy = Medium, preselection = generic, keypoint limit = 40000, tiepoint limit = 4000, constrain features by mask = 0
photo 0: 13163 points
photo 1: 14946 points
photo 2: 15278 points
photo 3: 15976 points
photo 4: 13906 points
photo 5: 13817 points
photo 6: 14868 points
photo 7: 14217 points
photo 8: 15481 points
photo 9: 14095 points
photo 10: 14487 points
photo 11: 15634 points
photo 12: 15616 points
photo 13: 16644 points
photo 14: 15868 points
photo 15: 17100 points
photo 16: 19168 points
photo 17: 18876 points
photo 18: 17147 points
photo 19: 15490 points
photo 20: 13756 points
photo 21: 14139 points
photo 22: 14010 points
photo 23: 14375 points
photo 24: 15793 points
photo 25: 16865 points
photo 26: 19149 points
photo 27: 19831 points
photo 28: 16872 points
photo 29: 13727 points
photo 30: 12739 points
photo 31: 13404 points
points detected in 9.52669 sec
29051 matche

## Export dense cloud
If you want to export the dense cloud for use in other software, you can run the code boxes below. If you want to just continue with the next step in the photogrammetry process, you can skip it.

In [125]:
# Quinn - Im guessing at these extenstions, I have only used oc3
pointsFormatExt = { 'PointsFormatOBJ':'.obj', 'PointsFormatPLY':'.ply', 'PointsFormatXYZ':'.xyz', 'PointsFormatLAS':'.las', 'PointsFormatExpe':'.???',
'PointsFormatU3D':'.u3d', 'PointsFormatPDF':'.pdf', 'PointsFormatE57':'.e57', 'PointsFormatOC3':'.oc3', 'PointsFormatPotree':'.???',
'PointsFormatLAZ':'.laz', 'PointsFormatCL3':'.cl3', 'PointsFormatPTS':'.pts', 'PointsFormatDXF':'.dxf' }

ptsfchoice=widgets.Dropdown(description='Points Format', value='PointsFormatOC3', options= ['PointsFormatOBJ', 'PointsFormatPLY', 'PointsFormatXYZ', 'PointsFormatLAS', 'PointsFormatExpe',
'PointsFormatU3D', 'PointsFormatPDF', 'PointsFormatE57', 'PointsFormatOC3', 'PointsFormatPotree',
'PointsFormatLAZ', 'PointsFormatCL3', 'PointsFormatPTS', 'PointsFormatDXF']   )


In [37]:
interact(chc, txt=ptsfchoice)

'PointsFormatOBJ'

In [38]:
template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
doc = PhotoScan.app.document \n\
doc.open(\"{}\") \n\
chunk = doc.chunk \n\
chunk.exportPoints(path=\"{}\", format=PhotoScan.PointsFormat.{})  \n' 

output = template.format(singularitymountfolder + projectFile,  singularitymountfolder + 'photoscandemo' + pointsFormatExt[ptsfchoice.value], ptsfchoice.value)

print('output:', output)

with open(commandScript, 'w') as f:
    f.write(output)

output: #!/usr/bin/env python3 
import PhotoScan 
doc = PhotoScan.app.document 
doc.open("/scratch//monument.psx") 
chunk = doc.chunk 
chunk.exportPoints(path="/scratch/photoscandemo.obj", format=PhotoScan.PointsFormat.PointsFormatOBJ)  



In [39]:
out = !sbatch slurmscript.sh   
        
print (' Export the points  output: ', out ) 
jobId =  out[0].split()[3]
print (jobId)

 Export the points  output:  ['Submitted batch job 1406450']
1406450


In [40]:
# print the users queue and the job status by id
!squeue -u $username
print('--------------------------------')
!scontrol show job $jobId

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1406450     savio monument   quinnd  R       0:03      1 n0044.savio1
--------------------------------
JobId=1406450 JobName=monument
   UserId=quinnd(42413) GroupId=ucb(501) MCS_label=N/A
   Priority=544271 Nice=0 Account=ac_scsguest QOS=savio_normal
   JobState=RUNNING Reason=None Dependency=(null)
   Requeue=0 Restarts=0 BatchFlag=1 Reboot=0 ExitCode=0:0
   RunTime=00:00:03 TimeLimit=01:00:00 TimeMin=N/A
   SubmitTime=2017-06-10T07:12:03 EligibleTime=2017-06-10T07:12:03
   StartTime=2017-06-10T07:12:04 EndTime=2017-06-10T08:12:04 Deadline=N/A
   PreemptTime=None SuspendTime=None SecsPreSuspend=0
   Partition=savio AllocNode:Sid=ln000:21556
   ReqNodeList=(null) ExcNodeList=(null)
   NodeList=n0044.savio1
   BatchHost=n0044.savio1
   NumNodes=1 NumCPUs=20 NumTasks=1 CPUs/Task=1 ReqB:S:C:T=0:0:*:*
   TRES=cpu=20,mem=62.50G,node=1
   Socks/Node=* NtasksPerN:B:S:C=0:0:*:* CoreSpec=*
   MinCPU

## Update project on Box
After generating the dense cloud, the .psx file will be much larger and this may take a moment.
**Maurice: add success message**

In [126]:
update_project_file_in_box()

find_folder_id folder_name:  monument
find_folder_id folderlist:  [<Box Folder - 27697792189 (monument)>]
Box folder id:  27697792189
project file list:  [<Box File - 183058668349 (monument.psx)>]
datazip file list:  [<Box File - 183058591009 (monument.files.zip)>]
project file id:  183058668349
data zip file list:  183058591009
begin project file update.
update psx result:  <Box File - 183058668349>    update zip result:  <Box File - 183058591009>


## OFFLINE: Dense cloud clean-up
At this point, you may want to clean up the dense point cloud by removing stray points (e.g. for objects that have not been photographed in a lightbox, where there might be artifacts from the background, etc.) This should be done using the desktop version of Photoscan. 

Be sure to save the updated project file back to the same place in Box if you want to run the next processing steps through this notebook.

# Build Mesh

First, retrieve the project file (.psx) from Box, in case it has been updated.

In [42]:
retrieve_project_file()

find_folder_id folder_name:  monument
find_folder_id folderlist:  [<Box Folder - 27697792189 (monument)>]
downloading:  monument.files.zip
data zip file download complete.
downloading:  monument.psx
project file download complete.


## Choose parameters
Running the code box below will create three sets of drop-downs where you can choose the surface type, face count, and data source.

In [43]:
stchoice=widgets.Dropdown(description='Surface type', value='Arbitrary', options=['Arbitrary', 'HeightField'])
fcchoice=widgets.Dropdown(description='Face count', value='HighFaceCount', options= ['LowFaceCount', 'MediumFaceCount', 'HighFaceCount'] )
dschoice=widgets.Dropdown(description='Data source', value='DenseCloudData', options=['PointCloudData', 'DenseCloudData', 'DepthMapsData', 'ModelData', 'TiledModelData', 'ElevationData',
'OrthomosaicData']  )


In [45]:
interact(chc, txt=stchoice)
interact(chc, txt=fcchoice)
interact(chc, txt=dschoice)

'DenseCloudData'

<function __main__.chc>

In [46]:
template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
import time \n\
print( "start time: ", time.time()) \n\
doc = PhotoScan.app.document \n\
doc.open(\"{}\") \n\
chunk = doc.chunk \n\
chunk.buildModel(surface=PhotoScan.{}, source=PhotoScan.{}, face_count=PhotoScan.{} ) \n\
doc.save(\"{}\") \n\
print( "stop time: ", time.time()) \n'

output = template.format(singularitymountfolder + projectFile, stchoice.value, dschoice.value, fcchoice.value, singularitymountfolder + projectFile)

print('output:', output)

with open(commandScript, 'w') as f:  
    f.write(output)


#set time limit for this batch run
outputbatchscript = batchtemplate.format('00:30:00')
with open(slurmScript, 'w') as f:  
    f.write(outputbatchscript)
    

output: #!/usr/bin/env python3 
import PhotoScan 
import time 
print( "start time: ", time.time()) 
doc = PhotoScan.app.document 
doc.open("/scratch//monument.psx") 
chunk = doc.chunk 
chunk.buildModel(surface=PhotoScan.Arbitrary, source=PhotoScan.DenseCloudData, face_count=PhotoScan.MediumFaceCount ) 
doc.save("/scratch//monument.psx") 
print( "stop time: ", time.time()) 



## Run mesh build
The code box below will submit your job and print out your job ID.

In [47]:
out = !sbatch slurmscript.sh   
    
print ('Execute the mesh build output: ', out ) 
jobId =  out[0].split()[3]
print (jobId) 

Execute the mesh build output:  ['Submitted batch job 1406451']
1406451


## Check on mesh build status
Run the code box below to get the current status of the mesh build job. You can run it multiple times for the most recent status.

In [50]:
# print the users queue and the job status by id
!squeue -u $username
print('--------------------------------')
!scontrol show job $jobId

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
--------------------------------
JobId=1406451 JobName=monument
   UserId=quinnd(42413) GroupId=ucb(501) MCS_label=N/A
   Priority=544271 Nice=0 Account=ac_scsguest QOS=savio_normal
   JobState=FAILED Reason=NonZeroExitCode Dependency=(null)
   Requeue=0 Restarts=0 BatchFlag=1 Reboot=0 ExitCode=1:0
   RunTime=00:02:37 TimeLimit=00:30:00 TimeMin=N/A
   SubmitTime=2017-06-10T07:13:19 EligibleTime=2017-06-10T07:13:19
   StartTime=2017-06-10T07:13:20 EndTime=2017-06-10T07:15:57 Deadline=N/A
   PreemptTime=None SuspendTime=None SecsPreSuspend=0
   Partition=savio AllocNode:Sid=ln000:21556
   ReqNodeList=(null) ExcNodeList=(null)
   NodeList=n0044.savio1
   BatchHost=n0044.savio1
   NumNodes=1 NumCPUs=20 NumTasks=0 CPUs/Task=1 ReqB:S:C:T=0:0:*:*
   TRES=cpu=20,mem=62.50G,node=1
   Socks/Node=* NtasksPerN:B:S:C=0:0:*:* CoreSpec=*
   MinCPUsNode=1 MinMemoryNode=62.50G MinTmpDiskNode=0
   Features=(null) Gres=(

## Display log message

As an optional step, you can run the code below to see the log file once the job is done.

In [51]:
# print the log file if necessary
slurmlogfilename = 'slurm-' + jobId + '.out'
with open(slurmlogfilename, 'r') as fin:
    print(fin.read(), end="")

Activation successful
start time:  1497104001.9384043
LoadProject
loaded project in 0.078207 sec
BuildModel: surface type = Arbitrary, source data = Dense cloud, face count = Medium, interpolation = Enabled
Grid size: 29219 x 26962 x 9084
Tree depth: 15
Tree set in 10.8818s (1357815 points)
Leaves/Nodes: 10386055/11869777
Laplacian constraints set in 3.06303s
Depth[0/15]: 1
	Evaluated / Got / Solved in: 0 / 5.81741e-05 / 6.58035e-05
Depth[1/15]: 8
	Evaluated / Got / Solved in: 0 / 0.000167131 / 0.00329208
Depth[2/15]: 64
	Evaluated / Got / Solved in: 0 / 0.000571012 / 0.004843
Depth[3/15]: 512
	Evaluated / Got / Solved in: 0 / 0.00102901 / 0.0106981
Depth[4/15]: 4096
	Evaluated / Got / Solved in: 0 / 0.00439477 / 0.0287468
Depth[5/15]: 32768
	Evaluated / Got / Solved in: 0 / 0.0482392 / 0.0571988
Depth[6/15]: 11640
	Evaluated / Got / Solved in: 0 / 0.00913405 / 0.0859375
Depth[7/15]: 24024
	Evaluated / Got / Solved in: 0 / 0.0197394 / 0.154392
Depth[8/15]: 58360
	Evaluated / Got / Solv

## Update project on Box
The next code box will return the project file to Box so you can make additional tweaks and changes offline.

In [52]:
update_project_file_in_box()

find_folder_id folder_name:  monument
find_folder_id folderlist:  [<Box Folder - 27697792189 (monument)>]
Box folder id:  27697792189
project file list:  [<Box File - 183058668349 (monument.psx)>]
datazip file list:  [<Box File - 183058591009 (monument.files.zip)>]
project file id:  183058668349
data zip file list:  183058591009
begin project file update.
update psx result:  <Box File - 183058668349>    update zip result:  <Box File - 183058591009>


## OFFLINE: Mesh cleanup
If you need to add marker points in the Photoscan file (.psx), download the latest project file from Box, make the changes, and upload the edited .psx file back to Box.

# Build Texture

First, retrieve the project file from Box in case it has been updated.

In [ ]:
retrieve_project_file()

## Choose parameters
Choose the data source, and specify a coordinate system if desired, after running the next code box.

In [53]:
gmchoice=widgets.Dropdown(description='UV Mapping Mode', value='GenericMapping', options= ['GenericMapping', 'OrthophotoMapping', 'AdaptiveOrthophotoMapping', 'SphericalMapping',
'CameraMapping'])
ccchoice=widgets.Dropdown(description='Color Correction', value='True', options= ['True', 'False'] )
ds2choice=widgets.Dropdown(description='Data source', value='ModelData', options=['PointCloudData', 'DenseCloudData', 'DepthMapsData', 'ModelData', 'TiledModelData', 'ElevationData',
'OrthomosaicData']  )

In [54]:
interact(chc, txt=gmchoice)
interact(chc, txt=ccchoice)
interact(chc, txt=ds2choice)

'ModelData'

<function __main__.chc>

In [55]:
template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
import time \n\
print( "start time: ", time.time()) \n\
doc = PhotoScan.app.document \n\
doc.open(\"{}\") \n\
chunk = doc.chunk \n\
chunk.buildUV(mapping=PhotoScan.{}) \n\
chunk.buildTexture( color_correction={} ) \n\
doc.save(\"{}\") \n\
doc.open(\"{}\") \n\
chunk = doc.chunk \n\
chunk.buildOrthomosaic( surface=PhotoScan.DataSource.{} ) \n\
doc.save(\"{}\") \n\
print( "stop time: ", time.time()) \n'

output = template.format(singularitymountfolder + projectFile, gmchoice.value, ccchoice.value,  singularitymountfolder + projectFile, singularitymountfolder + projectFile, ds2choice.value, singularitymountfolder + projectFile)

print('output:', output)

with open(commandScript, 'w') as f:  
    f.write(output)


output: #!/usr/bin/env python3 
import PhotoScan 
import time 
print( "start time: ", time.time()) 
doc = PhotoScan.app.document 
doc.open("/scratch//monument.psx") 
chunk = doc.chunk 
chunk.buildUV(mapping=PhotoScan.GenericMapping) 
chunk.buildTexture( color_correction=True ) 
doc.save("/scratch//monument.psx") 
doc.open("/scratch//monument.psx") 
chunk = doc.chunk 
chunk.buildOrthomosaic( surface=PhotoScan.DataSource.ModelData ) 
doc.save("/scratch//monument.psx") 
print( "stop time: ", time.time()) 



## Run texture build

In [56]:
out = !sbatch slurmscript.sh   
    
print ('  texture and orthomosaic build output: ', out ) 
jobId =  out[0].split()[3]
print (jobId)    

  texture and orthomosaic build output:  ['Submitted batch job 1406452']
1406452


In [59]:
# print the users queue and the job status by id
!squeue -u $username
print('--------------------------------')
!scontrol show job $jobId

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
--------------------------------
JobId=1406452 JobName=monument
   UserId=quinnd(42413) GroupId=ucb(501) MCS_label=N/A
   Priority=482814 Nice=0 Account=ac_scsguest QOS=savio_normal
   JobState=FAILED Reason=NonZeroExitCode Dependency=(null)
   Requeue=0 Restarts=0 BatchFlag=1 Reboot=0 ExitCode=1:0
   RunTime=00:02:21 TimeLimit=00:30:00 TimeMin=N/A
   SubmitTime=2017-06-10T07:18:11 EligibleTime=2017-06-10T07:18:11
   StartTime=2017-06-10T07:18:11 EndTime=2017-06-10T07:20:32 Deadline=N/A
   PreemptTime=None SuspendTime=None SecsPreSuspend=0
   Partition=savio AllocNode:Sid=ln000:21556
   ReqNodeList=(null) ExcNodeList=(null)
   NodeList=n0044.savio1
   BatchHost=n0044.savio1
   NumNodes=1 NumCPUs=20 NumTasks=0 CPUs/Task=1 ReqB:S:C:T=0:0:*:*
   TRES=cpu=20,mem=62.50G,node=1
   Socks/Node=* NtasksPerN:B:S:C=0:0:*:* CoreSpec=*
   MinCPUsNode=1 MinMemoryNode=62.50G MinTmpDiskNode=0
   Features=(null) Gres=(

## Update project on Box

In [60]:
update_project_file_in_box()

find_folder_id folder_name:  monument
find_folder_id folderlist:  [<Box Folder - 27697792189 (monument)>]
Box folder id:  27697792189
project file list:  [<Box File - 183058668349 (monument.psx)>]
datazip file list:  [<Box File - 183058591009 (monument.files.zip)>]
project file id:  183058668349
data zip file list:  183058591009
begin project file update.
update psx result:  <Box File - 183058668349>    update zip result:  <Box File - 183058591009>


# Export results

## Export the orthomosaic

In [61]:
# QUINN guessing at some of these extensions
ifextension = {'ImageFormatJPEG':'.jpg', 'ImageFormatTIFF':'.tiff', 'ImageFormatPNG':'.png', 'ImageFormatBMP':'.bmp', 'ImageFormatEXR':'.exr',
'ImageFormatPNM':'.pnm', 'ImageFormatSGI':'.sgi', 'ImageFormatCR2':'.cr2', 'ImageFormatSEQ':'.seq', 'ImageFormatARA':'.ara'}

ifchoice=widgets.Dropdown(description='Image Format', value='ImageFormatTIFF', options=['ImageFormatJPEG', 'ImageFormatTIFF', 'ImageFormatPNG', 'ImageFormatBMP', 'ImageFormatEXR',
'ImageFormatPNM', 'ImageFormatSGI', 'ImageFormatCR2', 'ImageFormatSEQ', 'ImageFormatARA'] )

In [62]:
interact(chc, txt=ifchoice)

'ImageFormatJPEG'

In [63]:
template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
doc = PhotoScan.app.document \n\
doc.open(\"{}\") \n\
chunk = doc.chunk \n\
chunk.exportOrthomosaic(path=\"{}\", image_format=PhotoScan.ImageFormat.{},)  \n' 

orthoFile = projectname + ifextension[ifchoice.value]
output = template.format(singularitymountfolder + projectFile,  singularitymountfolder + orthoFile, ifchoice.value)

print('output:', output)

with open(commandScript, 'w') as f:
    f.write(output)


output: #!/usr/bin/env python3 
import PhotoScan 
doc = PhotoScan.app.document 
doc.open("/scratch//monument.psx") 
chunk = doc.chunk 
chunk.exportOrthomosaic(path="/scratch/monument.jpg", image_format=PhotoScan.ImageFormat.ImageFormatJPEG,)  



In [64]:
out = !sbatch slurmscript.sh   
        
print (' Export the orthomosaic format output: ', out ) 
jobId =  out[0].split()[3]
print (jobId)  

 Export the orthomosaic format output:  ['Submitted batch job 1406454']
1406454


In [65]:
# print the users queue and the job status by id
!squeue -u $username
print('--------------------------------')
!scontrol show job $jobId

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           1406454     savio monument   quinnd  R       0:11      1 n0044.savio1
--------------------------------
JobId=1406454 JobName=monument
   UserId=quinnd(42413) GroupId=ucb(501) MCS_label=N/A
   Priority=435720 Nice=0 Account=ac_scsguest QOS=savio_normal
   JobState=RUNNING Reason=None Dependency=(null)
   Requeue=0 Restarts=0 BatchFlag=1 Reboot=0 ExitCode=0:0
   RunTime=00:00:12 TimeLimit=00:30:00 TimeMin=N/A
   SubmitTime=2017-06-10T07:23:27 EligibleTime=2017-06-10T07:23:27
   StartTime=2017-06-10T07:23:28 EndTime=2017-06-10T07:53:28 Deadline=N/A
   PreemptTime=None SuspendTime=None SecsPreSuspend=0
   Partition=savio AllocNode:Sid=ln000:21556
   ReqNodeList=(null) ExcNodeList=(null)
   NodeList=n0044.savio1
   BatchHost=n0044.savio1
   NumNodes=1 NumCPUs=20 NumTasks=1 CPUs/Task=1 ReqB:S:C:T=0:0:*:*
   TRES=cpu=20,mem=62.50G,node=1
   Socks/Node=* NtasksPerN:B:S:C=0:0:*:* CoreSpec=*
   MinCPU

## Move the orthomosaic file to Box

In [73]:
newFolderId = find_folder_id(boxProjectFolder)
upload_folder = client.folder(folder_id=newFolderId).get()
orthoUploaded = upload_folder.upload( runFolder + orthoFile)  
print ("orthomosaic file id: ", orthoUploaded['id'] )

find_folder_id folder_name:  monument
find_folder_id folderlist:  [<Box Folder - 27697792189 (monument)>]
orthomosaic file id:  183711291332


## Generate the Model export


In [74]:

# QUINN guessing at some of these extensions
mfextension = { 'ModelFormatOBJ':'.obj', 'ModelFormat3DS':'.3ds', 'ModelFormatVRML':'.vrml', 'ModelFormatPLY':'.ply', 'ModelFormatCOLLADA':'.col',
'ModelFormatU3D':'.u3d', 'ModelFormatPDF':'.pdf', 'ModelFormatDXF':'.dxf', 'ModelFormatFBX':'.fbx', 'ModelFormatKMZ':'.kmz',
'ModelFormatCTM':'.ctm', 'ModelFormatSTL':'.stl', 'ModelFormatDXF_3DF':'.3df', 'ModelFormatTLS':'.tls' }

mfchoice=widgets.Dropdown(description='Model Format', value='ModelFormatOBJ', options= ['ModelFormatOBJ', 'ModelFormat3DS', 'ModelFormatVRML', 'ModelFormatPLY', 'ModelFormatCOLLADA',
'ModelFormatU3D', 'ModelFormatPDF', 'ModelFormatDXF', 'ModelFormatFBX', 'ModelFormatKMZ',
'ModelFormatCTM', 'ModelFormatSTL', 'ModelFormatDXF_3DF', 'ModelFormatTLS'] )

In [75]:
interact(chc, txt=mfchoice)

'ModelFormatOBJ'

<function __main__.chc>

In [69]:
template = '#!/usr/bin/env python3 \n\
import PhotoScan \n\
doc = PhotoScan.app.document \n\
doc.open(\"{}\") \n\
chunk = doc.chunk \n\
chunk.exportModel(path=\"{}\", format=PhotoScan.ModelFormat.{})  \n' 

modelFile = projectname + mfextension[mfchoice.value]
output = template.format(singularitymountfolder + projectFile, singularitymountfolder + modelFile, mfchoice.value)
print('output:', output)

with open(commandScript, 'w') as f:
    f.write(output)


output: #!/usr/bin/env python3 
import PhotoScan 
doc = PhotoScan.app.document 
doc.open("/scratch//monument.psx") 
chunk = doc.chunk 
chunk.exportModel(path="/scratch/monument.obj", format=PhotoScan.ModelFormat.ModelFormatOBJ)  



In [70]:
out = !sbatch slurmscript.sh   
        
print (' Generate model output: ', out ) 
jobId =  out[0].split()[3]
print (jobId)         


 Generate model output:  ['Submitted batch job 1406456']
1406456


In [71]:
# print the users queue and the job status by id
!squeue -u $username
print('--------------------------------')
!scontrol show job $jobId

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
--------------------------------
JobId=1406456 JobName=monument
   UserId=quinnd(42413) GroupId=ucb(501) MCS_label=N/A
   Priority=435720 Nice=0 Account=ac_scsguest QOS=savio_normal
   JobState=FAILED Reason=NonZeroExitCode Dependency=(null)
   Requeue=0 Restarts=0 BatchFlag=1 Reboot=0 ExitCode=1:0
   RunTime=00:00:03 TimeLimit=00:30:00 TimeMin=N/A
   SubmitTime=2017-06-10T07:24:32 EligibleTime=2017-06-10T07:24:32
   StartTime=2017-06-10T07:24:32 EndTime=2017-06-10T07:24:35 Deadline=N/A
   PreemptTime=None SuspendTime=None SecsPreSuspend=0
   Partition=savio AllocNode:Sid=ln000:21556
   ReqNodeList=(null) ExcNodeList=(null)
   NodeList=n0044.savio1
   BatchHost=n0044.savio1
   NumNodes=1 NumCPUs=20 NumTasks=0 CPUs/Task=1 ReqB:S:C:T=0:0:*:*
   TRES=cpu=20,mem=62.50G,node=1
   Socks/Node=* NtasksPerN:B:S:C=0:0:*:* CoreSpec=*
   MinCPUsNode=1 MinMemoryNode=62.50G MinTmpDiskNode=0
   Features=(null) Gres=(

## Move the OBJ and associated JPG to Box
You can download the files from Box and preview using http://3dviewer.net/

In [77]:
newFolderId = find_folder_id(boxProjectFolder)
upload_folder = client.folder(folder_id=newFolderId).get()
objUploaded = upload_folder.upload(runFolder + modelFile)  
print ("obj file id: ", objUploaded['id'] )
jpgUploaded = upload_folder.upload(runFolder + jpgFile)  
print ("jpeg file id: ", jpgUploaded['id'] )

find_folder_id folder_name:  monument
find_folder_id folderlist:  [<Box Folder - 27697792189 (monument)>]
obj file id:  183711388797
jpeg file id:  183711415771


# Software license
This software is available under the terms of the Educational Community License, Version 2.0 (ECL 2.0). This software is Copyright 2016 The Regents of the University of California, Berkeley ("Berkeley").

The text of the ECL license is reproduced below.

Educational Community License, Version 2.0
*************************************
Copyright 2016 The Regents of the University of California, Berkeley ("Berkeley")

Educational Community License, Version 2.0, April 2007

The Educational Community License version 2.0 ("ECL") consists of the
Apache 2.0 license, modified to change the scope of the patent grant in
section 3 to be specific to the needs of the education communities using
this license. The original Apache 2.0 license can be found at:[http://www.apache.org/licenses/LICENSE-2.0]